In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
import time

In [3]:

data_dir = "/content/drive/MyDrive/farm_insects_project/data/processed/farm_insects"


train_dir = f"{data_dir}/splits/train"
val_dir = f"{data_dir}/splits/val"
test_dir = f"{data_dir}/splits/test"


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(val_dir, transform=val_test_transforms)
test_dataset  = datasets.ImageFolder(test_dir, transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)
print("Number of training images:", len(train_dataset))


Classes: ['Africanized Honey Bees (Killer Bees)', 'Aphids', 'Armyworms', 'Brown Marmorated Stink Bugs', 'Cabbage Loopers', 'Citrus Canker', 'Colorado Potato Beetles', 'Corn Borers', 'Corn Earworms', 'Fall Armyworms', 'Fruit Flies', 'Spider Mites', 'Thrips', 'Tomato Hornworms', 'Western Corn Rootworms']
Number of training images: 808


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
for param in vgg16.features.parameters():
    param.requires_grad = False


vgg16.classifier[6] = nn.Linear(4096, num_classes)

vgg16 = vgg16.to(device)
print("Model modified for", num_classes, "classes")


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier[6].parameters(), lr=1e-4)


num_epochs = 20

for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {avg_loss:.4f}")

vgg16.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print("\n Final Evaluation Metrics:")
print(f"Overall Accuracy:  {accuracy:.4f}")
print(f"Overall Precision: {precision:.4f}")
print(f"Overall Recall:    {recall:.4f}")
print(f"Overall F1-score:  {f1:.4f}")


Using device: cpu
Model modified for 15 classes
Epoch [1/20] - Training Loss: 2.6994
Epoch [2/20] - Training Loss: 2.2913
Epoch [3/20] - Training Loss: 1.9945
Epoch [4/20] - Training Loss: 1.7823
Epoch [5/20] - Training Loss: 1.6334
Epoch [6/20] - Training Loss: 1.5382
Epoch [7/20] - Training Loss: 1.4377
Epoch [8/20] - Training Loss: 1.3638
Epoch [9/20] - Training Loss: 1.3291
Epoch [10/20] - Training Loss: 1.2442
Epoch [11/20] - Training Loss: 1.2022
Epoch [12/20] - Training Loss: 1.1603
Epoch [13/20] - Training Loss: 1.1372
Epoch [14/20] - Training Loss: 1.1125
Epoch [15/20] - Training Loss: 1.0665
Epoch [16/20] - Training Loss: 1.0394
Epoch [17/20] - Training Loss: 1.0159
Epoch [18/20] - Training Loss: 0.9706
Epoch [19/20] - Training Loss: 0.9920
Epoch [20/20] - Training Loss: 0.9533

 Final Evaluation Metrics:
Overall Accuracy:  0.6245
Overall Precision: 0.6324
Overall Recall:    0.6245
Overall F1-score:  0.6158


In [6]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


googlenet = models.googlenet(weights=models.GoogLeNet_Weights.IMAGENET1K_V1)

# Freeze all layers first
for param in googlenet.parameters():
    param.requires_grad = False

# Unfreeze the last Inception block and the final classifier
for param in googlenet.inception5b.parameters():
    param.requires_grad = True

googlenet.fc = nn.Linear(1024, num_classes)

googlenet = googlenet.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, googlenet.parameters()), lr=1e-4)

num_epochs = 20

for epoch in range(num_epochs):
    googlenet.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = googlenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {avg_loss:.4f}")


googlenet.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = googlenet(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print("\n Final Performance Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


Using device: cpu
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


100%|██████████| 49.7M/49.7M [00:00<00:00, 136MB/s]


Epoch [1/20] - Training Loss: 2.6183
Epoch [2/20] - Training Loss: 2.3058
Epoch [3/20] - Training Loss: 2.0690
Epoch [4/20] - Training Loss: 1.8761
Epoch [5/20] - Training Loss: 1.7164
Epoch [6/20] - Training Loss: 1.5816
Epoch [7/20] - Training Loss: 1.4382
Epoch [8/20] - Training Loss: 1.3409
Epoch [9/20] - Training Loss: 1.2568
Epoch [10/20] - Training Loss: 1.1882
Epoch [11/20] - Training Loss: 1.1130
Epoch [12/20] - Training Loss: 1.0404
Epoch [13/20] - Training Loss: 1.0028
Epoch [14/20] - Training Loss: 0.9657
Epoch [15/20] - Training Loss: 0.8827
Epoch [16/20] - Training Loss: 0.8468
Epoch [17/20] - Training Loss: 0.7893
Epoch [18/20] - Training Loss: 0.7631
Epoch [19/20] - Training Loss: 0.7293
Epoch [20/20] - Training Loss: 0.6817

 Final Performance Metrics:
Accuracy : 0.6482
Precision: 0.6577
Recall   : 0.6482
F1 Score : 0.6461
